## DFJ file

#### Import packages

In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [ ]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

#### Assigning key variables

In [ ]:
# This is the database where the familyman data is stored on the platform
db1 = "familyman_dev_v3"
# This database is used for FCCSQ processing
fcsq_db = "fcsq"
# create path for within athena FCSQ database in the S3 folder, alongside other S3 items
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_disps/"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
#Select the snapshot date you require. Each record in Familyman has a snapshot date (essentially just a column in the tables) that relates to the database at that point in time
snapshot_date = "2022-08-04"

### Getting data from applications datasets

In [ ]:
#Fetching the data from the orders applied for dataset created in the main children act application processing
pydb.create_temp_table(
f""" 
SELECT 
  year,
  quarter,
  CASE WHEN order_case_type = 'P'
        THEN 'Private Law'
      WHEN order_case_type = 'C'
        THEN 'Public Law'
      ELSE 'check' END
    AS Category,
  COUNT(*) AS Start
FROM
  {fcsq_db}.ca_apps_order_types
GROUP BY
  year,
  quarter,
  order_case_type
  
""",
    
"ca_dfj_starts")

In [ ]:
pydb.create_temp_table(
f""" 
SELECT 
  year,
  quarter,
  CASE WHEN order_case_type = 'P'
        THEN 'Private Law'
      WHEN order_case_type = 'C'
        THEN 'Public Law'
      ELSE 'check' END
    AS Category,
  COUNT(*) AS Finish
FROM
  {fcsq_db}.ca_all_disposals
WHERE
  disp_type_code = 1
GROUP BY
  year,
  quarter,
  order_case_type
  
""",
    
"ca_dfj_end")

In [ ]:
pydb.read_sql_query("select * from __temp__.ca_dfj_starts where year > 2011 order by year, quarter")